In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string
import torch
import torch.nn as nn

In [3]:
def findFiles(path):
    return glob.glob(path)

In [4]:
print(findFiles("data/names/*.txt"))

['data/names/Italian.txt', 'data/names/Arabic.txt', 'data/names/English.txt', 'data/names/German.txt', 'data/names/French.txt', 'data/names/Spanish.txt', 'data/names/Greek.txt', 'data/names/Dutch.txt', 'data/names/Korean.txt', 'data/names/Indian.txt', 'data/names/Portuguese.txt', 'data/names/Japanese.txt', 'data/names/Polish.txt', 'data/names/Irish.txt', 'data/names/Chinese.txt', 'data/names/Russian.txt', 'data/names/Czech.txt', 'data/names/Vietnamese.txt', 'data/names/Scottish.txt']


In [5]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)
print(n_letters)
print(all_letters)

57
abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,;'


In [6]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [7]:
category_lines = {}
all_categories = []

In [8]:
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

In [9]:
print(all_categories)

['Italian', 'Arabic', 'English', 'German', 'French', 'Spanish', 'Greek', 'Dutch', 'Korean', 'Indian', 'Portuguese', 'Japanese', 'Polish', 'Irish', 'Chinese', 'Russian', 'Czech', 'Vietnamese', 'Scottish']


In [10]:
# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor


# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])
torch.Size([5, 1, 57])


In [11]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [12]:
input = lineToTensor('Alvia')
hidden = torch.zeros(1, n_hidden)
output, next_hidden = rnn(input[0], hidden)
print(output)

tensor([[-2.9710, -2.9072, -2.9783, -3.0594, -2.8567, -2.9229, -2.8978,
         -2.9429, -2.9012, -3.0249, -2.9325, -2.8865, -2.9660, -2.9117,
         -2.8867, -2.9579, -2.9808, -2.9868, -2.9979]])


In [13]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i.item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))
print(all_categories)

('French', 4)
['Italian', 'Arabic', 'English', 'German', 'French', 'Spanish', 'Greek', 'Dutch', 'Korean', 'Indian', 'Portuguese', 'Japanese', 'Polish', 'Irish', 'Chinese', 'Russian', 'Czech', 'Vietnamese', 'Scottish']


In [14]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = Portuguese / line = Pinho
category = Arabic / line = Kassis
category = Scottish / line = Duncan
category = Arabic / line = Handal
category = Chinese / line = Huan
category = Irish / line = Flanagan
category = Russian / line = Dozortsev
category = Italian / line = Farro
category = Indian / line = Shyam
category = Korean / line = Yi


In [15]:
criterion = nn.NLLLoss()

In [16]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

In [17]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

5000 5% (0m 46s) 2.8642 Machado / Italian ✗ (Portuguese)
10000 10% (1m 32s) 2.3072 Bergfalk / German ✓
15000 15% (2m 19s) 2.5594 Mudra / Indian ✗ (Czech)
20000 20% (3m 5s) 0.9554 Kikkert / Dutch ✓
25000 25% (3m 50s) 1.1336 Nahodkin / Russian ✓
30000 30% (4m 37s) 3.1431 Bischoffs / Greek ✗ (German)
35000 35% (5m 23s) 1.0399 Slapnickova / Czech ✓
40000 40% (6m 9s) 2.7578 Toru / Vietnamese ✗ (Japanese)
45000 45% (6m 55s) 3.7740 Donoghue / Irish ✗ (English)
50000 50% (7m 42s) 2.1119 Snider / German ✗ (Dutch)
55000 55% (8m 28s) 0.9231 Wildgrube / German ✓
60000 60% (9m 14s) 2.3892 Lowe / French ✗ (German)
65000 65% (10m 1s) 1.3818 Kwang  / Korean ✓
70000 70% (10m 48s) 0.9470 Mccallum / Scottish ✓
75000 75% (11m 33s) 1.9949 Ankur / Arabic ✗ (Indian)
80000 80% (12m 18s) 0.7921 Lau / Vietnamese ✗ (Chinese)
85000 85% (13m 4s) 2.5656 Jordan / Scottish ✗ (Polish)
90000 90% (13m 50s) 0.0537 Bellandini / Italian ✓
95000 95% (14m 35s) 0.1833 Shalashilin / Russian ✓
100000 100% (15m 19s) 3.2735 Ahear

In [18]:
# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output


In [32]:
def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, all_categories[category_index]))
            predictions.append([value, all_categories[category_index]])

predict("Grethel")


> Grethel
(-0.98) German
(-1.62) Czech
(-2.03) Indian
